<a href="https://www.kaggle.com/code/aisuko/summarization-nlp?scriptVersionId=164686222" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Overview

Summarization creates a shorter version of a document or an article that captures all the important information. Along with translation, it is another example of a task that can be formulated as a sequence-to-sequence task. Summarization can be:

**Extractive:** extract the most relevant information from a document

**Abstractive:** generative new text that captures the mose relevant information

We are going to fine-tune the pretrained label with `Translation` on the California state bill subset of the BillSum dataset for abstrative summarization.

In [1]:
%%capture
!pip install transformers==4.35.2
!pip install datasets==2.15.0
!pip install evaluate==0.4.1
!pip install rouge-score==0.1.2

In [2]:
import os
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

login(token=user_secrets.get_secret("HUGGINGFACE_TOKEN"))

os.environ["WANDB_API_KEY"]=user_secrets.get_secret("WANDB_API_KEY")
os.environ["WANDB_PROJECT"] = "Fine-tuning Distilbert(t5)"
os.environ["WANDB_NOTES"] = "Fine tune model distilbert base uncased"
os.environ["WANDB_NAME"] = "ft-t5-with-dill-sum"

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Load BillSum dataset

Start by loading the smaller California state bill subset of the BillSum dataset.

In [3]:
from datasets import load_dataset

billsum=load_dataset("billsum", split="ca_test")
print(billsum)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 1237
})


In [4]:
billsum=billsum.train_test_split(test_size=0.2)
print(billsum)

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 989
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 248
    })
})


In [5]:
billsum["train"][0]

{'text': 'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 1253.7 is added to the Health and Safety Code, to read:\n1253.7.\n(a) For purposes of this chapter, “observation services” means outpatient services provided by a general acute care hospital and that have been ordered by a provider, to those patients who have unstable or uncertain conditions potentially serious enough to warrant close observation, but not so serious as to warrant inpatient admission to the hospital. Observation services may include the use of a bed, monitoring by nursing and other staff, and any other services that are reasonable and necessary to safely evaluate a patient’s condition or determine the need for a possible inpatient admission to the hospital.\n(b) When a patient in an inpatient unit of a hospital or in an observation unit, as defined in subdivision (c), is receiving observation services, or following a change in a patient’s status from inpatient to observation, t

There are two fileds that you will want to use:

* `text` the text of the bill which will be the input to the model
* `summary` a condensed version of `text` which will be the model target

# Preprocess

The next step is to load a T5 tokenizer to process `text` and `summary`:

In [6]:
from transformers import AutoTokenizer

model_name="t5-small"
tokenizer=AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

The preprocessing function we want to create needs to:

* Prefix the input with a prompt so T5 knows this is a summarization task. Some model capable of multiple NLP tasks require prompting for specific tasks.

* Use the keyword `text_target` argument when tokenizing labels.

* Truncate sequences to be no longer than the maximum length set by the `max_length` parameter.

In [7]:
prefix="summarize:"

def preprocess_function(examples):
    inputs=[prefix+doc for doc in examples["text"]]
    model_inputs=tokenizer(inputs, max_length=1024, truncation=True)
    
    labels=tokenizer(text_target=examples["summary"], max_length=128, truncation=True)
    
    model_inputs["labels"]=labels["input_ids"]
    return model_inputs

In [8]:
tokenized_billsum=billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

In [9]:
from transformers import DataCollatorForSeq2Seq

data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_name)

# Evaluate

Including a metric during training is often helpful for evaluating our model's performance. Here we are going to load the ROUGE metric.


## ROUGE

Recall Oriented Understudy for Gisting Evaluation, is a set of metrics and software package used for evaluating automatic summarization and machine translation software in natural language processing. The metrics compare an automatically produced summary or translation against a reference of a set of references(human_produced) summary or translation.

In [10]:
import evaluate

rouge=evaluate.load("rouge")

Then create a function that passes your predictions and labels to `compute` to calcualte the ROUGE metric

In [11]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels=eval_pred
    decoded_preds=tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels=np.where(labels!=-100, labels, tokenizer.pad_token_id)
    decoded_labels=tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    result=rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    
    prediction_lens=[np.count_nonzero(pred!=tokenizer.pad_token_id) for pred in predictions]
    
    result["gen_len"]=np.mean(prediction_lens)
    
    return {k: round(v,4) for k,v in result.items()}

# Training

In [12]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model=AutoModelForSeq2SeqLM.from_pretrained(model_name)
print(model)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [13]:
training_args=Seq2SeqTrainingArguments(
    output_dir=os.getenv("WANDB_NAME"),
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16, # decrease from 16 to 8 to adapte low memory GPU
    per_device_eval_batch_size=16,
    gradient_checkpointing=True,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
    report_to="wandb",
    run_name=os.getenv("WANDB_NAME"),
    push_to_hub=False,
)


trainer=Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

wandb: Currently logged in as: urakiny (causal_language_trainer). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.16.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240228_113221-6qx2dwcd
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ft-t5-with-dill-sum
wandb: ⭐️ View project at https://wandb.ai/causal_language_trainer/Fine-tuning%20Distilbert%28t5%29
wandb: 🚀 View run at https://wandb.ai/causal_language_trainer/Fine-tuning%20Distilbert%28t5%29/runs/6qx2dwcd
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/conda/lib/python3.10/site-packages/

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,3.088087,0.139500,0.043600,0.115500,0.115400,19.000000
2,No log,2.757417,0.131500,0.039400,0.109000,0.109300,19.000000
3,No log,2.628376,0.137700,0.046500,0.113800,0.113900,19.000000
4,No log,2.579599,0.142800,0.051200,0.117700,0.117500,19.000000
5,No log,2.565781,0.144100,0.052600,0.118400,0.118400,19.000000


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=155, training_loss=3.1945511356476817, metrics={'train_runtime': 619.7407, 'train_samples_per_second': 7.979, 'train_steps_per_second': 0.25, 'total_flos': 1338530416558080.0, 'train_loss': 3.1945511356476817, 'epoch': 5.0})

# Evaluate

In [14]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 13.01


In [15]:
kwargs={
    'model_name': f'{os.getenv("WANDB_NAME")}',
    'finetuned_from': model_name,
    'tasks': 'summarization',
    'dataset_tags':'bills-summarization',
    'dataset':'billsum'
}

tokenizer.push_to_hub(os.getenv("WANDB_NAME"))
trainer.push_to_hub(**kwargs)

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.28k [00:00<?, ?B/s]

'https://huggingface.co/aisuko/ft-t5-with-dill-sum/tree/main/'

# Inference

In [16]:
from transformers import pipeline

text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."

summarizer=pipeline("summarization", model=os.getenv("WANDB_NAME"))
summarizer(text)

Your max_length is set to 200, but your input_length is only 103. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=51)


[{'summary_text': "the Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs . it's the most aggressive action on tackling the climate crisis in American history . no one making under $400,000 per year will pay a penny more in taxes."}]

## Using PyTorch

In [17]:
from transformers import AutoTokenizer

tokenizer=AutoTokenizer.from_pretrained(os.getenv("WANDB_NAME"))
inputs=tokenizer(text, return_tensors="pt").input_ids

In [18]:
from transformers import AutoModelForSeq2SeqLM

model=AutoModelForSeq2SeqLM.from_pretrained(os.getenv("WANDB_NAME"))
outputs=model.generate(inputs, max_new_tokens=100, do_sample=False)

In [19]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

"the Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. it's the most aggressive action on tackling the climate crisis in American history. it'll ask the ultra-wealthy and corporations to pay their fair share."